# Phenology in S2 NDVI



**Prerequisites**
* In this notebook we are using openEO to fetch the time series data for the meadow. You can register for a free trial account on the [openEO Platform](https://openeo.cloud/#plans) website.

Lets start with importing the different libraries that we need within this notebook.

In [3]:
import os
import sys

# Add local version of FuseTS
module_path = os.path.abspath(os.path.join('../../src'))
if module_path not in sys.path:
    sys.path.append(module_path)


import numpy as np
import openeo
import xarray
from fusets.analytics import phenology



%matplotlib inline

ModuleNotFoundError: No module named 'openeo'

### Downloading the S2 time series
In order to execute the peak valley algorithm, we need to have time series data. Retrieving time series data can be done through various methods, and one such method is using openEO. [OpenEO](https://openeo.org/) is an API that provides access to a variety of Earth Observation (EO) data and processing services in a standardized and easy-to-use way. By leveraging the power of openEO, we can easily retrieve the time series data for the meadow and use it to analyze the patterns and trends.

More information on the usage of openEO's Python client can be found on [GitHub](https://github.com/Open-EO/openeo-python-client).

The first step is to connect to an openEO compatible backend.

In [2]:
connection = openeo.connect("openeo.vito.be").authenticate_oidc()

Authenticated using refresh token.


Next we define the area of interest, in this case an extent, for which we would like to fetch time series data.

In [3]:
minx, miny, maxx, maxy = (15.179421073198585, 45.80924633589998, 15.185336903822831, 45.81302555710934)
spat_ext = dict(west=minx, east=maxx, north=maxy, south=miny, crs=4326)
temp_ext = ["2021-01-01", "2021-12-31"]

We will create an openEO process to calculate the NDVI time series for our area of interest. We'll begin by using the SENTINEL2_L2A_SENTINELHUB collection, and apply a cloud masking algorithm to remove any interfering clouds before calculating the NDVI values.

In [4]:
s2 = connection.load_collection('SENTINEL2_L2A_SENTINELHUB',
                                spatial_extent=spat_ext,
                                temporal_extent=temp_ext,
                                bands=["B04","B08","SCL"])
s2 = s2.process("mask_scl_dilation", data=s2, scl_band_name="SCL")
ndvi_cube = s2.ndvi(red="B04", nir="B08", target_band='NDVI')

Now that we have calculated the NDVI time series for our area of interest, we can request openEO to download the result to our local storage. This will allow us to access the file and use it for further analysis in this notebook. However, if we have already downloaded the file, we can use the existing time series to continue our analysis without the need for a new download.

In [5]:
if not os.path.exists('./s2_meadow.nc'):
    job = ndvi_cube.execute_batch("s2_meadow.nc", title=f'AI4FOOD - Peak Valley - Time Series', out_format="netCDF")

ds = xarray.load_dataset('./s2_meadow.nc')
ndvi = ds.NDVI

Now that we have calculated the NDVI time series, we can utilize it to execute the peak valley algorithm that is part of the FuseTS algorithm. The peak valley algorithm is a powerful tool that allows us to detect significant changes in the vegetation patterns over time. 

In [6]:
pv_result = peakvalley(ndvi, drop_thr=0.13, rec_r = 1.0, slope_thr=-0.007)

In [10]:

kdims = list(ndvi.dims)

lon = hv.Dimension("lon", label="longitude", unit="deg Northing")
lat = hv.Dimension("lat", label="latitude", unit="deg Easting")

hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d' # gets rid of the time dimension in the slider

gv_ndvi = gv.Dataset(ndvi, kdims=kdims, vdims=["ndvi"], crs=ccrs.epsg("32633")).redim(x=lon, y=lat)
ndvi_map = gv_ndvi.to(gv.Image, ["lon", "lat"], "ndvi", group="ndvi", label="raw", datatype=["xarray"], dynamic=True)
img_ndvi = ndvi_map.redim(x=lon, y=lat)

overlay = OSM() * img_ndvi
overlay.opts(
    opts.Image(cmap="viridis", colorbar=True, clim=(0, 1), alpha=0.8, tools=["hover"]),
    opts.Tiles(height=300, width=500))

dim_t = hv.Dimension("t", label="time")
dim_index = hv.Dimension("norm ndvi", label="norm ndvi")

def get_curve_and_vspan(x, y):
    x2 = (abs(x) // 10)*10 + 5
    y2 = (abs(y) // 10)*10 + 5
    out = hv.Curve(ndvi.sel(x=x2,y=y2).dropna(dim='t'), dim_t, dim_index, label="Raw timeseries").opts(color='#D3D3D3')
    
    data = pv_result.sel(x=x2, y=y2)
    pairs = np.transpose([np.where(data == 1)[0], np.where(data == -1)[0]])

    
    out *= hv.VSpan(pv_result.t.values[0], pv_result.t.values[1]).opts(color='red', alpha=0.0)
    if len(pairs) == 0: 
        return out
    
    for pair in pairs:
        out *= hv.VSpan(pv_result.t.values[pair[0]], pv_result.t.values[pair[1]]).opts(color='red', alpha=0.2)
    
    return out

clicker = streams.Tap(source=img_ndvi, x=ndvi["x"].values[0], y=ndvi["y"].values[0]) 
aux_map = hv.DynamicMap(callback=get_curve_and_vspan, streams=[clicker])

layout = overlay + aux_map
layout.opts(
    opts.Image(cmap="RdYlGn", colorbar=True, clim=(0, 1), tools=["hover"], alpha=0.8),
    opts.Tiles(height=300, width=400),
    opts.Curve(height=300, width=400, ylim=(0, 1)))

lat


BokehModel(combine_events=True, render_bundle={'docs_json': {'746a8254-ead8-43e5-b23f-85a09cf665ed': {'defs': …

:Layout
   .DynamicMap.I  :DynamicMap   [t]
      :Overlay
         .Tiles.I  :Tiles   [x,y]
         .Ndvi.Raw :Image   [lon,lat]   (ndvi)
   .DynamicMap.II :DynamicMap   []
      :Overlay
         .Curve.Raw_timeseries :Curve   [t]   (norm ndvi)
         .VSpan.I              :VSpan   [x,y]
         .VSpan.II             :VSpan   [x,y]
         .VSpan.III            :VSpan   [x,y]